In [1]:
import pandas as pd
import numpy as np

#### Binary Models creation (dummy data)

In [3]:
n = 1000
series_list = []
headers = ['real'] + [f'model_{i}' for i in range(0, 10)]
data_hashmap = {}
for header in headers:
    data_hashmap[header] = list(np.random.choice(['x', 'y'], size=n))

df = pd.DataFrame.from_dict(data_hashmap)
df.head(20)


,real,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,model_9
0,y,y,x,y,y,x,y,x,x,y,y
1,x,y,y,x,x,x,x,x,x,y,x
2,x,y,x,y,y,x,y,y,x,y,y
3,y,x,x,x,y,x,x,x,y,y,x
4,x,y,y,y,x,x,y,y,y,x,y
5,y,x,x,x,x,x,y,y,x,x,y
6,y,y,y,y,x,x,y,x,y,y,x
7,y,y,y,y,x,x,x,y,y,x,y
8,x,y,y,x,x,x,x,x,x,x,x
9,y,x,y,x,y,y,x,x,y,x,x


In [5]:
for i, _ in enumerate(headers[1:]):
    df[f'accuracy_{i}'] = np.where(df['real'] == df[f'model_{i}'], 1, 0)

df[['real', 'model_0', 'accuracy_0', 'model_1', 'accuracy_1', 'model_9', 'accuracy_9']].head(10)

,real,model_0,accuracy_0,model_1,accuracy_1,model_9,accuracy_9
0,y,y,1,x,0,y,1
1,x,y,0,y,0,x,1
2,x,y,0,x,1,y,0
3,y,x,0,x,0,x,0
4,x,y,0,y,0,y,0
5,y,x,0,x,0,y,1
6,y,y,1,y,1,x,0
7,y,y,1,y,1,y,1
8,x,y,0,y,0,x,1
9,y,x,0,y,1,x,0


#### create Xs and Ys count for model

In [6]:
def create_value_counts_columns(df, col, values):
    counts = df[[col]].value_counts()
    for v in values:
        count_v = counts.get(v)
        df[f'{col}_count_{v}'] = count_v
    return df



#### Computing Precion, Recall, F1

In [7]:
def compute_precision_recall_f1(df, model):
    df = df.copy()
    df = create_value_counts_columns(df, 'real', ['x', 'y'])
    df = create_value_counts_columns(df, model, ['x', 'y'])
    df[f'{model}_x_accuracy'] = np.where((df['accuracy_0'] == 1) & (df[f'{model}'] == 'x'), 1, 0)
    df[f'{model}_y_accuracy'] = np.where((df['accuracy_0'] == 1) & (df[f'{model}'] == 'y'), 1, 0)

    sum_x_accuracy = int(df[[f'{model}_x_accuracy']].sum())
    sum_y_accuracy = int(df[[f'{model}_y_accuracy']].sum())

    df[f'{model}_x_accurate'] = sum_x_accuracy
    df[f'{model}_x_inaccurate'] = df[f'{model}_count_x'] - df[f'{model}_x_accurate']
    df[f'{model}_y_accurate'] = sum_y_accuracy
    df[f'{model}_y_inaccurate'] = df[f'{model}_count_y'] - df[f'{model}_y_accurate']
    df[f'{model}_observations_x'] = df[f'{model}_x_accurate'] + df[f'{model}_y_inaccurate']
    df[f'{model}_observations_y'] = df[f'{model}_y_accurate'] + df[f'{model}_x_inaccurate']

    precision_x, precision_y = f'precision_x', f'precision_y'
    recall_x, recall_y = f'recall_x', f'recall_y'
    f1_score_x, f1_score_y = 'f1_score_x', 'f1_score_y'
    df[precision_x] = df[f'{model}_x_accurate'] / df[f'{model}_observations_x']
    df[recall_x] = df[f'{model}_x_accurate'] / df[f'{model}_count_x']
    df[precision_y] = df[f'{model}_y_accurate'] / df[f'{model}_observations_y']
    df[recall_y] = df[f'{model}_y_accurate'] / df[f'{model}_count_y']

    df[f1_score_x] = (df[precision_x] * df[recall_x] * 2) / (df[precision_x] + df[recall_x])
    df[f1_score_y] = (df[precision_y] * df[recall_y] * 2) / (df[precision_y] + df[recall_y])

    df['precision(x+y)'] = (df[precision_x] + df[precision_y]) / 2
    df['recall(x+y)'] = (df[recall_x] + df[recall_y]) / 2
    df['f1_score(x+y)'] = (df[f1_score_x] + df[f1_score_y]) / 2

    return df

In [8]:
select_cols = ['precision_x', 'recall_x', 'f1_score_x', 'precision_y', 'recall_y', 'f1_score_y', 'precision(x+y)', 'recall(x+y)', 'f1_score(x+y)']

In [9]:
precision_and_recall = compute_precision_recall_f1(df, 'model_0')
precision_and_recall[select_cols].head(1)

,precision_x,recall_x,f1_score_x,precision_y,recall_y,f1_score_y,precision(x+y),recall(x+y),f1_score(x+y)
0,0.520792,0.515686,0.518227,0.50101,0.506122,0.503553,0.510901,0.510904,0.51089


In [10]:
precision_and_recall = compute_precision_recall_f1(df, 'model_1')
precision_and_recall[select_cols].head(1)

,precision_x,recall_x,f1_score_x,precision_y,recall_y,f1_score_y,precision(x+y),recall(x+y),f1_score(x+y)
0,0.521739,0.494118,0.507553,0.500967,0.528571,0.514399,0.511353,0.511345,0.510976


In [11]:
precision_and_recall = compute_precision_recall_f1(df, 'model_2')
precision_and_recall[select_cols].head(1)

,precision_x,recall_x,f1_score_x,precision_y,recall_y,f1_score_y,precision(x+y),recall(x+y),f1_score(x+y)
0,0.495868,0.494845,0.495356,0.525194,0.526214,0.525703,0.510531,0.510529,0.51053


In [12]:
precision_and_recall = compute_precision_recall_f1(df, 'model_3')
precision_and_recall[select_cols].head(1)

,precision_x,recall_x,f1_score_x,precision_y,recall_y,f1_score_y,precision(x+y),recall(x+y),f1_score(x+y)
0,0.500994,0.514286,0.507553,0.521127,0.507843,0.514399,0.51106,0.511064,0.510976
